<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_text_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

In [13]:
%%time

!git clone -l -s git://github.com/kavyajeetbora/EAST.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 80 (delta 43), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (80/80), 977.37 KiB | 10.86 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/cloned-repo/cloned-repo
dataset_original.py  dsiplay_ground_truth.ipynb  loss.py
dataset.py	     EAST_detection.ipynb	 model.py
detect.py	     EAST_text_detector.ipynb	 train.py
CPU times: user 41.6 ms, sys: 14.2 ms, total: 55.8 ms
Wall time: 3.75 s


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from dataset import *

In [41]:
# AI4Bharat dataset
image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Images'
label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Annotations'


# icdar dataset
# image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Images'
# label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Labels'

training_dataset = custom_dataset(image_address, label_address)
print(len(training_dataset))

428


## Dataset Initial Check

In [42]:
img = Image.open(image_address+'/5.jpg')

with open(label_address+'/5.txt', 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)
  
print(labels)

[ 40.00946874 458.48609436 323.43148912 417.38303403 320.61145491
 504.2269818   51.81025341 528.43789142]
--------------------
[341.69060663 393.53091057 660.68408924 326.73418052 685.64494953
 458.82473082 346.12011362 497.13160495]
--------------------
[1 1]


In [43]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(934.)

## Loss for one batch

In [44]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=8)

imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))

print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())

print(torch.sum(gt_score))

torch.Size([8, 3, 512, 512]) torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128]) torch.Size([8, 1, 128, 128])
tensor(11365.)
CPU times: user 4.15 s, sys: 4.22 s, total: 8.37 s
Wall time: 9.68 s


In [45]:
from model import EAST

model = EAST()

Downloading: "http://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:06<00:00, 81.2MB/s]


In [46]:
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128])


In [47]:
from loss import Loss

loss_fn = Loss()

loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.83342028, angle loss is 0.82373893, iou loss is 3.24717069


tensor(12.3180, grad_fn=<AddBackward0>)

## Training Full Batch 

fine tuning pretrained model [east_vgg16.pth](https://drive.google.com/file/d/1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg/view)

In [0]:
from model import EAST

model = EAST()
state_dict = torch.load('/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/model_epoch_20.pth')
model.load_state_dict(state_dict)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [0]:
def train(model, train_img_path, train_gt_path, pths_path, batch_size, lr, epoch_iter, num_workers=4, interval=5):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[epoch_iter//2], gamma=0.1)
  
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, os.path.join(pths_path, 'model_epoch_{}.pth'.format(epoch+1)))

In [71]:
train(model, image_address, label_address, 'models', batch_size=8, lr=1e-4, epoch_iter=30)

428


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


classify loss is 0.20035398, angle loss is 0.00281524, iou loss is 0.62110615
Epoch is [1/30], mini-batch is [1/53], time consumption is 1.01319623, batch_loss is 0.84961247
classify loss is 0.22705466, angle loss is 0.00231697, iou loss is 0.64552951
Epoch is [1/30], mini-batch is [2/53], time consumption is 0.93117595, batch_loss is 0.89575392
classify loss is 0.27056074, angle loss is 0.00923281, iou loss is 0.61577547
Epoch is [1/30], mini-batch is [3/53], time consumption is 0.76887059, batch_loss is 0.97866434
classify loss is 0.22516030, angle loss is 0.00991720, iou loss is 0.63095015
Epoch is [1/30], mini-batch is [4/53], time consumption is 0.76942444, batch_loss is 0.95528251
classify loss is 0.21793216, angle loss is 0.00296295, iou loss is 0.62771434
Epoch is [1/30], mini-batch is [5/53], time consumption is 0.79238510, batch_loss is 0.87527603
classify loss is 0.29522365, angle loss is 0.00679880, iou loss is 0.55263096
Epoch is [1/30], mini-batch is [6/53], time consumpt

NameError: ignored